## EE-361M Introduction to Data Mining
## Assignment #5
## Due: Thursday, Apr 14, 2016 by 2pm; Total points: 60


Your homework should be written in a **Jupyter notebook** (if this isn't possible, let me know). Please use this naming format for your notebook you submit: **Group(Group Num)_HW(HW Number).ipynb**. For example, Group1_HW1.ipynb. Homeworks should be submitted through Canvas in your **groups of 3 from the first homework**. If groups need to be adjusted please contact the TA. Also, please make sure your code runs and the graphics (and anything else) are displayed in your notebook before submitting.

# Question 1 (1+1+5+3+3+2=15pts) - Logistic Regression

In this question we will be predicting whether someone will have an affair! Yes - there is data on this. See below on how to import the data.
1. Convert naffairs to a binary variable hadAffair which is 1 if had an affair and zero otherwise
2. Split the data into training and test. Use 42 as random seed and use 1/3rd of the data for testing. Our y variable is hadAffair and X matrix includes all the other variables except naffairs.
3. Train a logistic regression with almost no regularization (pass l2 (ridge) to penalty and 1,000 to the C parameter which is the inverse of regularization strength lambda. This essentially does l2 regularization but applies very little weight to the penalty term) and report the confusion matrix on the test data. Also report the accuracy for the "no affairs" class, the affairs class, and the average per-class accuracy on the test data. Average per-class accuracy is described in this [post](http://blog.dato.com/how-to-evaluate-machine-learning-models-part-2a-classification-metrics).
4. Repeat step 3 except use l2 penalty with Cs of [.001, .01,0.1, 1]. You will want to use k-fold cross validation to select the best parameter. To evaluate which parameter is best, maximize the average per-class accuracy. To help with this task, check out [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and how to make your own [custom scorer](http://scikit-learn.org/stable/modules/model_evaluation.html).
5. Repeat question 4 except use l1 (i.e. Lasso) instead of l2 as the penalty type.
6. Which model produces the best average per-class accuracy? Why do you think this is the case? How do the models handle the different classes, and why is this so?

In [2]:
from pydataset import data
df = data('affairs')

# Question 2 (2+3+2+3=10pts) - Support Vector Classifier

This question will continue to use the data from question 1 - including the training and test split data.
1. Fit a support vector classifier using the standard options on [sklearn's SVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC). Report the confusion matrix on the test data. Also report the accuracy for the no affairs class, the affairs class, and the average per-class accuracy (same as question 1).
2. Repeat question 1 except use grid search to select the best value of C within this set: [0.001, 0.01, 0.1, 1,5,10,100] and try both a radial and polynomial kernel (thus trying 14 combinations). Choose the combination that maximizes the average per-class accuracy. Use 5 folds. Report the best model, the confusion matrix, the accuracy for the no affairs class, the affairs class, and the average per-class accuracy.
3. Briefly discuss the effect of different  C,  kernel combinations.
4. Discuss your results from parts 1 and 2 and mention how they differ from Question 1's results.

# Question 3 (2+1+3+1+3=10pts) - Regression Trees

This question is very similar to homework 4 question 1. Except now we will be using regression trees and not classification trees, and you will be addressing a regression problem (i.e., the independent variable "price" will not be binarized).

For this question, we will be using the housing dataset (see code below). 

1. Convert driveway, recroom, fullbase, gashw, airco, and prefarea to numeric dummy variables (1 for yes, zero for no)
2. Split the data into training and testing with a random seed of 42 and keeping 1/3rd of the data for testing
2. Fit a [decision tree regressor](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor) to predict price using all the data (your dummy variables plus bedrooms and bathrooms).
5. Report the root MSE on the test data.
6. How does the tree decide on a splitting point?

In [95]:
df = data('Housing')

# Question 4 (2+5+3=10pts) - Support Vector Regression

This question will continue to use the data from question 3 - including the training and test split data.

1. Fit a support vector regression using the standard options on [sklearn's SVR](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html). Report the root MSE.
2. Repeat question 1 except use grid search to select the best value of C within this set: [0.001, 0.01, 0.1, 1,5,10,100] and try both a radial and polynomial kernel (thus trying 14 combinations). Choose the combination that minimizes MSE. Use 5 folds. Report the best model and the test root MSE.
4. Discuss your results from parts 1 and 2 and how they differ from Question 3 results.

# Question 5 (3+2+2+2+2+2+2=15pts) - Random Forest

This question will also continue to use the data from Question 1.
1. Fit a random forest model grid searching over the following values: {'n_estimators':[10, 100, 1000], 'max_features': ['auto', 'sqrt', 'log2']}. Choose the combination that maximizes the average per-class accuracy. Use 5 folds. Report the best model, the confusion matrix, the accuracy for the no affairs class, the affairs class, and the average per-class accuracy.
2. What do the n_estimators and max_features parameters do?
3. Report the features in order of importance based on the model used in part 1
4. Repeat question 1 using an AdaBoostClassifier and grid search over the following values: {'n_estimators':[50, 500, 5000], 'learning_rate': [.001, .01, .1]}
5. What does the learning_rate parameter do?
6. Report the features in order of importance based on the model used in part 4
7. Compare the results in part 1 and 4 and questions 2 and 3

In [50]:
#5.1
from pydataset import data
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation, svm
from sklearn.metrics import confusion_matrix

def convert_dummy(col,row):
    if row[col] > 0:
        return 1
    else:
        return 0

df = data('affairs')
df['naffairs'] = df.apply(lambda row: convert_dummy('naffairs',row),axis=1)
X = df[['kids', 'vryunhap','unhap','avgmarr','hapavg','vryhap','antirel','notrel','slghtrel','smerel',
          'vryrel','yrsmarr1','yrsmarr2','yrsmarr3','yrsmarr4','yrsmarr5','yrsmarr6']].values
y = df.naffairs.values

clf = RandomForestClassifier()
param_grid = {
    'n_estimators': [10, 100, 1000],
    'max_features': ['auto', 'sqrt', 'log2']
}

rs = GridSearchCV(clf, param_grid=param_grid)
rs.fit(X,y)
print(rs.best_estimator_)
# 5.1 Best estimator is max_features=auto, n_estimator=1000
X_train,X_test,y_train,y_test = cross_validation.train_test_split(
     X, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=1000, max_features='auto')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

naffair, nacorr, waffair, wacorr = 0, 0, 0, 0
for i in range(len(y_test)):
    if y_test[i] == 0:
        naffair = naffair + 1
        if y_pred[i] == 0:
            nacorr = nacorr + 1
    else:
        waffair = waffair + 1
        if y_pred[i] == 1:
            wacorr = wacorr + 1

naaccu = float(nacorr) / naffair
waaccu =  float(wacorr)/waffair
perclassaccu = (naaccu + waaccu) / 2
print("The confusion matrix is:\n", confusion_matrix(y_test,y_pred))
print("The accuracy for no affair class is:", naaccu)
print("The accuracy for affair class is:", waaccu)
print("The average per-class accuracy is:", perclassaccu)

importances = clf.feature_importances_
print(importances)
# 5.2
# N_estimators shows the number of trees in the forest.
# Max_features shows the number of features to consider when looking for the best split:
# 5.3
# From the weight table, we can see that features are ranked as follows in terms of weight:
# (from most to least important) kids, vryhap, antirel, unhap,  ... yrsmarr2, vryrel, yrsmarr1

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
The confusion matrix is:
 [[80 11]
 [25  5]]
The accuracy for no affair class is: 0.8791208791208791
The accuracy for affair class is: 0.16666666666666666
The average per-class accuracy is: 0.5228937728937729
[ 0.10683433  0.05587136  0.06854369  0.0522069   0.05583066  0.07528018
  0.06950056  0.0582785   0.06009735  0.05871943  0.03493494  0.03357845
  0.04579172  0.04823245  0.05820833  0.05633889  0.06175225]


In [54]:
#5.4
from pydataset import data
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn import cross_validation, svm
from sklearn.metrics import confusion_matrix

def convert_dummy(col,row):
    if row[col] > 0:
        return 1
    else:
        return 0

df = data('affairs')
df['naffairs'] = df.apply(lambda row: convert_dummy('naffairs',row),axis=1)
X = df[['kids', 'vryunhap','unhap','avgmarr','hapavg','vryhap','antirel','notrel','slghtrel','smerel',
          'vryrel','yrsmarr1','yrsmarr2','yrsmarr3','yrsmarr4','yrsmarr5','yrsmarr6']].values
y = df.naffairs.values

clf = AdaBoostClassifier()
param_grid = {
    'n_estimators': [50, 500, 5000],
    'learning_rate': [0.001, 0.01, 0.1]
}

rs = GridSearchCV(clf, param_grid=param_grid)
rs.fit(X,y)
print(rs.best_estimator_)
# 5.1 Best estimator is learning_rate = 0.001, n_estimator=5000

X_train,X_test,y_train,y_test = cross_validation.train_test_split(
     X, y, test_size=0.2, random_state=42)
clf = AdaBoostClassifier(n_estimators=5000, learning_rate=0.001)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

naffair, nacorr, waffair, wacorr = 0, 0, 0, 0
for i in range(len(y_test)):
    if y_test[i] == 0:
        naffair = naffair + 1
        if y_pred[i] == 0:
            nacorr = nacorr + 1
    else:
        waffair = waffair + 1
        if y_pred[i] == 1:
            wacorr = wacorr + 1

naaccu = float(nacorr) / naffair
waaccu =  float(wacorr)/waffair
perclassaccu = (naaccu + waaccu) / 2
print("The confusion matrix is:\n", confusion_matrix(y_test,y_pred))
print("The accuracy for no affair class is:", naaccu)
print("The accuracy for affair class is:", waaccu)
print("The average per-class accuracy is:", perclassaccu)

importances = clf.feature_importances_
print(importances)
# 5.5
# Learning rate shrinks the contribution of each classifier by learning_rate.
# It determines how much we learn from each new training data.
# 5.6
# From the weight table, we can see that features are ranked as follows in terms of weight:
# (from most to least important) kids, vryrel, yrsmarr2, unhap, vryunhap, antirel, vryhap,
# smerel, yrsmarr1, and the rest has no impact on the final result.

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.001, n_estimators=5000, random_state=None)
The confusion matrix is:
 [[88  3]
 [29  1]]
The accuracy for no affair class is: 0.967032967032967
The accuracy for affair class is: 0.03333333333333333
The average per-class accuracy is: 0.5001831501831502
[ 0.037   0.1432  0.1688  0.      0.      0.1358  0.1406  0.      0.      0.117
  0.0184  0.0624  0.1768  0.      0.      0.      0.    ]


In [ ]:
# 5.7 The two has similar average per-class accuracy, which is just slightly better than 50%
# However, Randomized Forest does better in determining affair class while adaboost does better
# in determining no-affair class.
# 